In [ ]:
# =============================================================
# 1️⃣ IMPORTS ET CONFIGURATION GÉNÉRALE
# =============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# --- chemins ---
data_path = "../data/"
clean_path = "../data/clean/"

# Crée le dossier clean s’il n’existe pas
os.makedirs(clean_path, exist_ok=True)

files = os.listdir(data_path)
print("📂 Fichiers détectés :", files)

# =============================================================
# 2️⃣ FONCTIONS DE NETTOYAGE GÉNÉRIQUES
# =============================================================

def normalize_cols(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df


def to_weekly(df, date_col, value_col, agg="mean"):
    """Convertit une colonne date en hebdomadaire"""
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])
    df["week"] = df[date_col].dt.to_period("W").apply(lambda r: r.start_time)
    df = df.groupby("week")[value_col].agg(agg).reset_index()
    df.rename(columns={"week": "date"}, inplace=True)
    return df


def regional_mean(df, region_col, value_col):
    """Calcule la moyenne nationale à partir des valeurs régionales"""
    return df.groupby(["date"])[value_col].mean().reset_index()

# =============================================================
# 3️⃣ LECTURE ET NETTOYAGE PERSONNALISÉ DES FICHIERS
# =============================================================

# --- A. Grippe : urgences France entière ---
grippe_fr = pd.read_csv(data_path + "grippe-passages-aux-urgences-et-actes-sos-medecins-france.csv", sep=";", low_memory=False)
grippe_fr = normalize_cols(grippe_fr)
grippe_fr = to_weekly(grippe_fr, date_col="date_de_passage", value_col="nombre_de_passages", agg="sum")
grippe_fr.rename(columns={"nombre_de_passages": "cas_grippe_fr"}, inplace=True)
grippe_fr.to_csv(clean_path + "grippe_fr_clean.csv", index=False)
print("✅ Grippe France nettoyé :", grippe_fr.shape)


# --- B. Grippe par région ---
grippe_reg = pd.read_csv(data_path + "grippe-passages-urgences-et-actes-sos-medecin_reg.csv", sep=";", low_memory=False)
grippe_reg = normalize_cols(grippe_reg)
if "date_de_passage" in grippe_reg.columns and "nombre_de_passages" in grippe_reg.columns:
    grippe_reg = to_weekly(grippe_reg, date_col="date_de_passage", value_col="nombre_de_passages", agg="sum")
    grippe_reg.rename(columns={"nombre_de_passages": "cas_grippe_reg"}, inplace=True)
grippe_reg.to_csv(clean_path + "grippe_reg_clean.csv", index=False)
print("✅ Grippe Région nettoyé :", grippe_reg.shape)


# --- C. Couverture vaccinale France ---
vacc_fr = pd.read_csv(data_path + "couvertures-vaccinales-des-adolescents-et-adultes-depuis-2011-france.csv", sep=";", low_memory=False)
vacc_fr = normalize_cols(vacc_fr)
if "annee" in vacc_fr.columns:
    vacc_fr["date"] = pd.to_datetime(vacc_fr["annee"].astype(str) + "-01-01")
    vacc_fr = vacc_fr.groupby("date")["couverture_vaccinale_pourcent"].mean().reset_index()
vacc_fr.to_csv(clean_path + "vacc_fr_clean.csv", index=False)
print("✅ Vaccination France nettoyé :", vacc_fr.shape)


# --- D. Couverture vaccinale Région ---
vacc_reg = pd.read_csv(data_path + "couvertures-vaccinales-des-adolescents-et-adultes-depuis-2011-region.csv", sep=";", low_memory=False)
vacc_reg = normalize_cols(vacc_reg)
if "annee" in vacc_reg.columns:
    vacc_reg["date"] = pd.to_datetime(vacc_reg["annee"].astype(str) + "-01-01")
    vacc_reg = vacc_reg.groupby("date")["couverture_vaccinale_pourcent"].mean().reset_index()
    vacc_reg.rename(columns={"couverture_vaccinale_pourcent": "couverture_vaccinale_region"}, inplace=True)
vacc_reg.to_csv(clean_path + "vacc_reg_clean.csv", index=False)
print("✅ Vaccination Région nettoyé :", vacc_reg.shape)


# --- E. Pharmacies ---
pharma = pd.read_csv(data_path + "santefr-lieux-vaccination-grippe-pharmacie.csv", sep=";", low_memory=False)
pharma = normalize_cols(pharma)
if "departement" in pharma.columns:
    df_pharma = pharma.groupby("departement").size().reset_index(name="nb_pharmacies")
    nb_pharma_total = df_pharma["nb_pharmacies"].sum()
else:
    nb_pharma_total = len(pharma)
df_pharma.to_csv(clean_path + "pharma_clean.csv", index=False)
print("✅ Pharmacies nettoyé :", nb_pharma_total)

# =============================================================
# 4️⃣ FUSION DES SOURCES NETTOYÉES
# =============================================================

df = grippe_fr.merge(vacc_fr, on="date", how="left")
df = df.merge(vacc_reg, on="date", how="left")

df["nb_pharmacies"] = nb_pharma_total
df = df.sort_values("date").reset_index(drop=True)

print("✅ Données fusionnées :", df.shape)
df.head()

# =============================================================
# 5️⃣ NETTOYAGE FINAL ET FEATURES
# =============================================================

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(method="ffill")

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)

for lag in [1, 2, 3, 4]:
    df[f"cas_grippe_fr_lag{lag}"] = df["cas_grippe_fr"].shift(lag)

df = df.dropna().reset_index(drop=True)
print("✅ Dataset final prêt :", df.shape)

# =============================================================
# 6️⃣ SÉPARATION TRAIN (2023) / TEST (2024)
# =============================================================

train_df = df[df["year"] == 2023]
test_df = df[df["year"] == 2024]

X_train = train_df.drop(columns=["cas_grippe_fr", "date"])
y_train = train_df["cas_grippe_fr"]

X_test = test_df.drop(columns=["cas_grippe_fr", "date"])
y_test = test_df["cas_grippe_fr"]

print("📊 Jeu d'entraînement :", X_train.shape, "| Test :", X_test.shape)

# =============================================================
# 7️⃣ RANDOM FOREST — PRÉDICTION 2024
# =============================================================

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=1
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"✅ Prédiction 2024 effectuée :")
print(f"   ➤ MAE : {mae:.2f}")
print(f"   ➤ R²  : {r2:.3f}")

plt.figure(figsize=(10,4))
plt.plot(test_df["date"], y_test, label="Réel", color="orange")
plt.plot(test_df["date"], y_pred, label="Prédit", color="green")
plt.title("📈 Prédiction du nombre de cas de grippe (2024)")
plt.legend()
plt.grid(True)
plt.show()

# =============================================================
# 8️⃣ SAUVEGARDE DU DATASET NETTOYÉ
# =============================================================

output_path = clean_path + "data_clean.csv"
df.to_csv(output_path, index=False)
print(f"💾 Dataset propre sauvegardé : {output_path}")

# =============================================================
# 9️⃣ ANALYSE DE CORRÉLATION ENTRE LES VARIABLES
# =============================================================

import seaborn as sns

print("🔍 Analyse de corrélation entre les variables principales...")

# --- On sélectionne uniquement les colonnes numériques
num_cols = df.select_dtypes(include=[np.number]).columns
corr = df[num_cols].corr()

# --- Affichage du tableau de corrélation
display(corr.round(2))

# --- Heatmap pour visualisation globale
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("🔗 Corrélation entre variables (plus proche de 1 ou -1 = forte corrélation)")
plt.show()

# =============================================================
# 🔁 Corrélation décalée dans le temps
# =============================================================

from pandas.plotting import lag_plot

def cross_corr(a, b, lag=0):
    """Calcule la corrélation croisée entre deux séries avec décalage."""
    return a.corr(b.shift(lag))

corrs = []
for lag in range(-6, 7):  # décalage de -6 à +6 semaines
    corrs.append({
        "lag(semaine)": lag,
        "corr(vaccin_vs_grippe)": cross_corr(df["couverture_vaccinale_pourcent"], df["cas_grippe_fr"], lag)
    })

corr_lags = pd.DataFrame(corrs)
corr_lags.plot(x="lag(semaine)", y="corr(vaccin_vs_grippe)", kind="bar", color="teal", legend=False)
plt.title("Corrélation vaccination ↔ cas grippe selon le décalage temporel")
plt.axhline(0, color="black", linewidth=0.8)
plt.ylabel("Corrélation")
plt.xlabel("Décalage (semaines)")
plt.show()

# =============================================================
# 🔍 Importance des variables selon le modèle Random Forest
# =============================================================

importances = pd.Series(rf.feature_importances_, index=X_train.columns)
importances = importances.sort_values(ascending=True).tail(15)

plt.figure(figsize=(8,6))
importances.plot(kind="barh", color="seagreen")
plt.title("💡 15 Variables les plus influentes — Random Forest")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()



📂 Fichiers détectés : ['doses-actes-2024.csv', 'inc-3-RDD-ds2.csv', 'couverture-2024.csv', 'santefr-lieux-vaccination-grippe-pharmacie.csv', 'Openhealth_S-Grippal_Regions.csv', 'couvertures-vaccinales-des-adolescents-et-adultes-depuis-2011-france.csv', 'grippe-passages-aux-urgences-et-actes-sos-medecins-departement.csv', 'inc-3-PAY-ds2.csv', 'grippe-passages-urgences-et-actes-sos-medecin_reg.csv', 'campagne-2024.csv', 'clean', 'grippe-passages-aux-urgences-et-actes-sos-medecins-france.csv', 'couvertures-vaccinales-des-adolescent-et-adultes-departement.csv', 'couvertures-vaccinales-des-adolescents-et-adultes-depuis-2011-region.csv', 'Openhealth_S-Grippal.csv']


KeyError: 'date_de_passage'